# Hashtag Treinamentos - Projeto: Automação Web

# - Módulo 32: Projeto 2 - Busca de Preços

### Objetivo: treinar um projeto em que a gente tenha que usar automações web com Selenium para buscar as informações que precisamos

- Já fizemos um projeto com esse objetivo no Módulo de Python e Web, mas não custa nada treinar mais um pouco.

### Como vai funcionar:

- Imagina que você trabalha na área de compras de uma empresa e precisa fazer uma comparação de fornecedores para os seus insumos/produtos.

- Nessa hora, você vai constantemente buscar nos sites desses fornecedores os produtos disponíveis e o preço, afinal, cada um deles pode fazer promoção em momentos diferentes e com valores diferentes.

- Seu objetivo: descobrir o produto mais barato e atualizar isso em uma planilha.
- Caso o valor seja 20% (ou mais) abaixo do preço original, queremos também ser avisados por e-mail para poder agir rápido e aproveitar essa promoção.

- No nosso caso, vamos fazer com produtos comuns em sites como Amazon, Magazine Luiza e Lojas Americanas, mas a ideia é a mesma.

### Outra opção:

- APIs

### O que temos disponível?

- Planilha de Produtos, com os nomes dos produtos e o link em cada loja, além do preço original cadastrado.

### O que devemos fazer:

- Cadastrar na coluna Preço Atual o menor preço encontrado e na Coluna Local o nome do Local onde foi encontrado esse preço
- Enviar um e-mail para compras com a notificação do menor preço encontrado e o link de compra, caso o preço encontrado esteja com 20% ou mais de desconto em relação ao preço original. (Vou usar o e-mail pythonimpressionador+compras@gmail.com. Use um e-mail seu para fazer os testes para ver se a mensagem está chegando)

### Adicional:

- Podemos colocar esse programa para rodar de 4 em 4 horas ou então todo dia as 10hrs da manhã. Podemos fazer isso via agendador de tarefas do Windows ou então deixar o código rodando em background com um time.sleep(tempo)
- Caso queira, você pode deixar o navegador sem aparecer ao término do seu código, para ficar mais sutil o seu programa

In [43]:
# criar um navegador
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import pandas as pd

# criar o navegador
nav = webdriver.Chrome()

In [44]:
# importar/visualizar a base de dados
tabela_produtos = pd.read_excel(r"C:\Users\Daniel\Documents\Arquivos Técnicos\Arquivos\Programação\_Cursos\Python Impressionador - Hashtag Treinamento\32.Projeto 2 - Automação Web - Aplicação de Mercado de Trabalho  ------ATUALIZADO\MATERIAL DE APOIO\buscas.xlsx")
display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 11 64 gb,mini watch,3000,4500
1,iphone 12 64 gb,mini watch,3500,5500


### Definição das funções de busca no google e no buscape

In [17]:
import time

def busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    
    # 1) entrar no google
    nav.get("https://www.google.com/")
    
    # tratar os valores que vieram da tabela
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_produto = produto.split(" ")
    
    preco_maximo = float(preco_maximo)
    preco_minimo = float(preco_minimo)
    

    # 2) pesquisar o nome do produto no google
    nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(produto)
    nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

    # 3) clicar na aba shopping -> Para resolver casos de itens na página com mesmo nome
    elementos = nav.find_elements(By.CLASS_NAME, 'hdtb-mitem')
    for item in elementos:
        if "Shopping" in item.text:
            item.click()
            break

    # 4) pegar a lista de resultados da busca no google shopping
    lista_resultados = nav.find_elements(By.CLASS_NAME, 'sh-dgr__grid-result')
    
    # para cada resultado, ele vai verificar se o resultado corresponde a todas as nossas condicoes
    lista_ofertas = [] # lista que a função vai me dar como resposta
    for resultado in lista_resultados:
        nome = resultado.find_element(By.CLASS_NAME, 'Xjkr3b').text
        nome = nome.lower()

        # verificacao do nome - se no nome tem algum termo banido
        tem_termos_banidos = False
        for palavra in lista_termos_banidos:
            if palavra in nome:
                tem_termos_banidos = True
        
        # verificar se no nome tem todos os termos do nome do produto
        tem_todos_termos_produto = True
        for palavra in lista_termos_produto:
            if palavra not in nome:
                tem_todos_termos_produto = False

        if not tem_termos_banidos and tem_todos_termos_produto: # verificando o nome
            try: # Try: Tente pegar o preço, se der erro (continue).
                preco = resultado.find_element(By.CLASS_NAME, 'a8Pemb').text
                preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
                preco = float(preco)
                # verificando se o preco ta dentro do minimo e maximo
                if preco_minimo <= preco <= preco_maximo:
                    elemento_link = resultado.find_element(By.CLASS_NAME, 'aULzUe')
                    elemento_pai = elemento_link.find_element(By.XPATH, '..')
                    link = elemento_pai.get_attribute('href')
                    lista_ofertas.append((nome, preco, link)) # Adicione uma Tupla para usar o append: Coloque entre ()
            except:
                continue
            
            
    return lista_ofertas

In [22]:
# Teste:

produto = 'iphone 11 64 gb'
termos_banidos = 'mini watch usado seminovo enjoei'
preco_minimo = 2500
preco_maximo = 3500

lista_ofertas_google_shopping = busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo)
print(lista_ofertas_google_shopping)

[('iphone 11 64gb black ios 4g câmera 12mp - apple', 3285.0, 'https://www.google.com/url?url=https://temperclima.com.br/Iphone-11-64Gb-Black-iOS-4G-Camera-12Mp-Apple&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwji9t3P7fv3AhVBH7kGHfrSCYcQgOUECJoL&usg=AOvVaw1lPRj9YLLnkq2vdJ5a6Mi7'), ('iphone 11 64gb - vitrine 12x sem juros original com garantia apple', 3449.0, 'https://www.google.com/url?url=https://produto.mercadolivre.com.br/MLB-2640842969-iphone-11-64gb-vitrine-12x-sem-juros-original-com-garantia-_JM%3Fmatt_tool%3D18956390%26utm_source%3Dgoogle_shopping%26utm_medium%3Dorganic&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwji9t3P7fv3AhVBH7kGHfrSCYcQgOUECLgL&usg=AOvVaw1fi9_8LJ8TqbPAjfBF19cH'), ('apple iphone 11, 64 gb, roxo - desbloqueado (premium renovado)', 2865.0, 'https://www.google.com/url?url=https://tiendamia.com/br/producto%3Famz%3DB011SDYBZW&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwji9t3P7fv3AhVBH7kGHfrSCYcQguUECOcL&usg=AOvVaw1hjuIpixbtsuekwzg-FvGH'), ('verifique disponibilidade | celular iphone 11 64gb', 2750.0, 

In [40]:
def busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    # tratar os valores da função
    preco_maximo = float(preco_maximo)
    preco_minimo = float(preco_minimo)
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_produto = produto.split(" ")
    
    
    # entrar no buscape
    nav.get("https://www.buscape.com.br/")
    
    # pesquisar pelo produto no buscape
    nav.find_element(By.CLASS_NAME, 'AutoCompleteStyle_input__FInnF').send_keys(produto, Keys.ENTER)
    
    # pegar a lista de resultados da busca do buscape
    time.sleep(5)
    lista_resultados = nav.find_elements(By.CLASS_NAME, 'Cell_Content__fT5st')
    
    # para cada resultado
    lista_ofertas = []
    for resultado in lista_resultados:
        try:
            preco = resultado.find_element(By.CLASS_NAME, 'CellPrice_MainValue__JXsj_').text
            nome = resultado.get_attribute('title')
            nome = nome.lower()
            link = resultado.get_attribute('href')
            
            # verificacao do nome - se no nome tem algum termo banido
            tem_termos_banidos = False
            for palavra in lista_termos_banidos:
                if palavra in nome:
                    tem_termos_banidos = True  
                    
            # verificar se no nome tem todos os termos do nome do produto
            tem_todos_termos_produto = True
            for palavra in lista_termos_produto:
                if palavra not in nome:
                    tem_todos_termos_produto = False            
            
            if not tem_termos_banidos and tem_todos_termos_produto:
                preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
                preco = float(preco)
                if preco_minimo <= preco <= preco_maximo:
                    lista_ofertas.append((nome, preco, link))
        except:
            pass
    return lista_ofertas

In [41]:
# Teste:

produto = 'iphone 11 64 gb'
termos_banidos = 'mini watch'
preco_minimo = 2500
preco_maximo = 3800

lista_ofertas = busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo)
print(lista_ofertas)

[('smartphone apple iphone 11 64gb ios câmera dupla', 3599.0, 'https://www.buscape.com.br/celular/smartphone-apple-iphone-11-64gb-ios?_lc=88&searchterm=iphone%2011%2064%20gb'), ('smartphone apple iphone 11 vermelho 64gb ios câmera dupla', 3599.0, 'https://www.buscape.com.br/celular/smartphone-apple-iphone-11-vermelho-64gb-ios?_lc=88&searchterm=iphone%2011%2064%20gb'), ('iphone x prata 64gb tela 5.8 ios 11 4g wi-fi câmera 12mp apple', 3153.04, 'https://www.buscape.com.br/lead?oid=807622688&channel=86&index=18&searchterm=iphone%2011%2064%20gb'), ('iphone x prata 64gb tela 5.8 ios 11 4g wi-fi câmera 12mp', 3153.04, 'https://www.buscape.com.br/lead?oid=807622443&channel=86&index=19&searchterm=iphone%2011%2064%20gb'), ('iphone x prata 64gb tela 5.8 ios 11 4g wi-fi câmera 12mp', 3153.04, 'https://www.buscape.com.br/lead?oid=807619821&channel=86&index=20&searchterm=iphone%2011%2064%20gb'), ('iphone x prata 64gb tela 5.8" ios 11 4g wi-fi câmera 12mp - apple', 3153.04, 'https://www.buscape.com.

### Contrução da nossa lista de ofertas encontradas

In [45]:
tabela_ofertas = pd.DataFrame() # Criar uma tabela vazia

for linha in tabela_produtos.index:
    produto = tabela_produtos.loc[linha, "Nome"]
    termos_banidos = tabela_produtos.loc[linha, "Termos banidos"]
    preco_minimo = tabela_produtos.loc[linha, "Preço mínimo"]
    preco_maximo = tabela_produtos.loc[linha, "Preço máximo"]
    
    lista_ofertas_google_shopping = busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=['produto', 'preco', 'link'])
        tabela_ofertas = tabela_ofertas.append(tabela_google_shopping)
    else:
        tabela_google_shopping = None
        
    lista_ofertas_buscape = busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['produto', 'preco', 'link'])
        tabela_ofertas = tabela_ofertas.append(tabela_buscape)
    else:
        tabela_buscape = None

display(tabela_ofertas)

,produto,preco,link
0,iphone 11 64 gb - branco apple,3599.00,https://www.google.com/url?url=https://www.fas...
1,iphone 11 64gb black ios 4g câmera 12mp - apple,3285.00,https://www.google.com/url?url=https://temperc...
2,apple iphone 11 de 64 gb amarelo,3899.00,https://www.google.com/url?url=https://www.sub...
3,"vitrine: iphone 11 apple 64gb branco tela 6,1 ...",3749.00,https://www.google.com/url?url=https://www.car...
4,iphone 11 64gb - vitrine 12x sem juros origina...,3449.00,https://www.google.com/url?url=https://produto...
...,...,...,...
6,"iphone 12 preto, com tela de 6,1&quot;, 5g, 64...",4559.05,https://www.buscape.com.br/lead?oid=750005116&...
7,"iphone 12 preto, com tela de 6,1&quot;, 5g, 64...",4559.05,https://www.buscape.com.br/lead?oid=748031166&...
8,"iphone 12 apple verde, 64gb desbloqueado - mgj...",5485.81,https://www.buscape.com.br/lead?oid=746940947&...
9,"apple iphone 12 , 64gb - azul",5278.80,https://www.buscape.com.br/lead?oid=728776662&...


In [56]:
# Ordenando pelo menor preço
tabela_ofertas = tabela_ofertas.sort_values(by=['preco'], ascending=True)
display(tabela_ofertas)

,produto,preco,link
25,iphone 11 64 gb roxo | celular apple,3050.00,https://www.google.com/url?url=https://www.enj...
30,iphone 11 64 gb amarelo | celular apple,3050.00,https://www.google.com/url?url=https://www.enj...
17,iphone 11 64 gb | celular apple,3050.00,https://www.google.com/url?url=https://www.enj...
28,original apple iphone 11 64gb / purple,3057.00,https://www.google.com/url?url=https://www.dub...
20,original apple iphone 11 64gb / white,3057.00,https://www.google.com/url?url=https://www.dub...
...,...,...,...
19,iphone 12 vitrine / 64gb / preto,5350.00,https://www.google.com/url?url=https://www.wil...
10,"apple iphone 12 , 64gb - azul",5382.30,https://www.buscape.com.br/lead?oid=728776466&...
8,"iphone 12 apple verde, 64gb desbloqueado - mgj...",5485.81,https://www.buscape.com.br/lead?oid=746940947&...
17,"iphone 12 64gb vermelho, 5g, 6.1, 12mp - mgj73...",5499.00,https://www.google.com/url?url=https://www.mag...


### Exportar a base de ofertas para Excel

In [57]:
# exportar pro excel
tabela_ofertas = tabela_ofertas.reset_index(drop=True)
tabela_ofertas.to_excel("Ofertas.xlsx", index=False)

### Enviando o e-mail

In [ ]:
# enviar por e-mail o resultado da tabela
import win32com.client as win32

#verificando se existe alguma oferta dentro da tabela de ofertas
if len(tabela_ofertas.index) > 0:
    # vou enviar email
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = 'email@gmail.com'
    mail.Subject = 'Produto(s) Encontrado(s) na faixa de preço desejada'
    mail.HTMLBody = f"""
    <p>Prezados,</p>
    <p>Encontramos alguns produtos em oferta dentro da faixa de preço desejada. Segue tabela com detalhes</p>
    {tabela_ofertas.to_html(index=False)}
    <p>Qualquer dúvida estou à disposição</p>
    <p>Att.,</p>
    """
    
    mail.Send()

nav.quit()